In [124]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import numpy as np
import os
import re
from scipy import stats as st
from statsmodels.stats.proportion import proportions_ztest as ztest

In [45]:
plt.style.use('fivethirtyeight')
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.min_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', None)

# Test case #1

Assuming all files are identical in their structure being the segments of a larger chunk, we should be just fine with a loop concatenating the small portions back into one dataset. At four million rows we're unlikely to lack memory, however in such a case it's fairly easy to build in some data type optimization for each file.  
To make it easier we can put all the csv files into one folder created specifically for this purpose, and afterwards save the single file in case we need it later.

In [17]:
df = pd.DataFrame()

for file in os.scandir('csvs/'):
    print(f'Reading file: {file}')
    tmp = pd.read_csv(file)
    df = pd.concat([df, tmp])
    
df.info()

df.to_csv('the_whole_dataset.csv')

Reading file: <DirEntry 'test_csv - Copy (2).csv'>
Reading file: <DirEntry 'test_csv - Copy (3).csv'>
Reading file: <DirEntry 'test_csv - Copy (4).csv'>
Reading file: <DirEntry 'test_csv - Copy (5).csv'>
Reading file: <DirEntry 'test_csv - Copy (6).csv'>
Reading file: <DirEntry 'test_csv - Copy (7).csv'>
Reading file: <DirEntry 'test_csv - Copy (8).csv'>
Reading file: <DirEntry 'test_csv - Copy (9).csv'>
Reading file: <DirEntry 'test_csv - Copy.csv'>
Reading file: <DirEntry 'test_csv.csv'>
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5761020 entries, 0 to 576101
Data columns (total 6 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   HomeFname          object
 1   HomeLname          object
 2   Account            int64 
 3   HomeUserID         int64 
 4   HomeEventDatetime  object
 5   EventType          object
dtypes: int64(2), object(4)
memory usage: 307.7+ MB


# Test Case #2

## Results, before you read the rest

The first stab at this research was based on the assumption that past members contain the same valuable indicators as the ten users who did pay for a membership or a ticket.  

I compared the scarce numerical data and quantified several data points to look for measurable differences between the groups. The bottom line is I found next to nothing, and even the noticable difference in the number of subscribers which makes sense on paper, turns out to not be statistically significant.  

If you're still curious about my train of thought feel free to continue reading.

## First look

In [3]:
df = pd.read_excel('linkedin.xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 74 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   linkedinProfile               348 non-null    object        
 1   Result                        348 non-null    object        
 2   Other                         191 non-null    object        
 3   email                         216 non-null    object        
 4   description                   278 non-null    object        
 5   headline                      348 non-null    object        
 6   location                      348 non-null    object        
 7   firstName                     348 non-null    object        
 8   lastName                      348 non-null    object        
 9   fullName                      348 non-null    object        
 10  subscribers                   347 non-null    float64       
 11  userId                        28

In [4]:
df.head()

,linkedinProfile,Result,Other,email,description,headline,location,firstName,lastName,fullName,subscribers,userId,mutualConnectionsText,mailFromDropcontact,company,companyUrl,jobTitle,jobDescription,jobLocation,jobDateRange,company2,companyUrl2,jobTitle2,jobDescription2,jobDateRange2,school,schoolUrl,schoolDegree,schoolDegreeSpec,schoolDateRange,school2,schoolUrl2,schoolDegree2,schoolDegreeSpec2,schoolDateRange2,qualificationFromDropContact,civilityFromDropContact,phoneNumberFromDropContact,websiteFromDropContact,twitter,website,companyWebsite,allSkills,skill1,endorsement1,skill2,endorsement2,skill3,endorsement3,skill4,endorsement4,skill5,endorsement5,skill6,endorsement6,baseUrl,profileId,timestamp,jobLocation2,connectionsCount,mail,birthday,schoolDescription2,naf5CodeFromDropContact,naf5DesFromDropContact,sirenFromDropContact,siretFromDropContact,siretAddressFromDropContact,siretZipFromDropContact,vatFromDropContact,schoolDescription,nbEmployeesFromDropContact,Unnamed: 72,Unnamed: 73
0,https://www.linkedin.com/in/mark-kohoot-96a08b1,5. Purchased Membership,Hunter,mk@aeroscena.com,Mark founded Aeroscena in 2010 with the missio...,Bio-pharmaceutical / functional fragrance,"Cleveland, Ohio, United States",Mark,Kohoot,Mark Kohoot,1704.00,3172258.00,1 mutual connection: Eric Sullivan,mk@aeroscena.com,Aeroscena,https://www.linkedin.com/company/aeroscena/,CEO,Aeroscena develops and markets fragrances from...,"Cleveland/Akron, Ohio Area",Sep 2010 â€“ Present,Cleveland Clinic Wellness Institute,https://www.linkedin.com/company/cleveland-cli...,Advisor to the Chairman,Supporting the Chairman of the Wellness Instit...,2009 â€“ 2010,Audencia Nantes Ecole de Management,https://www.linkedin.com/school/19998/?legacyS...,master,marketing,1984 â€“ 1985,The Ohio State University,https://www.linkedin.com/school/19133/?legacyS...,BSBA,International Business,1979 â€“ 1983,nominative@pro,Mr,+1 800-671-1890,www.aeroscena.com,ShopAscents,stumbleupon.com/content/1LSMVB,https://aeroscena.com,"Strategic Planning, Leadership, Marketing Stra...",Strategic Planning,31,Leadership,26,Marketing Strategy,24,Strategy,21,New Business Development,18,Business Strategy,15,https://www.linkedin.com/in/mark-kohoot-96a08b1,mark-kohoot-96a08b1,2022-01-03T20:00:58.586Z,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.linkedin.com/in/jonbowerman,5. Purchased Membership,Hunter,jon@kidsdraw.org,A creative problem solver who can rapidly desi...,Co Founder and CTO - Product Development and M...,"Farmington, Michigan, United States",Jon,Bowerman,Jon Bowerman,1745.00,14910006.00,1 mutual connection: Eric Sullivan,jon@kidsdraw.org,Kids Draw Bob,https://www.linkedin.com/company/kidsdrawbob/,President and Illustrator,The energy in a room of children who've found ...,"Bloomfield Hills, MI",Jul 2019 â€“ Present,"High Level Marketing, LLC",https://www.linkedin.com/company/high-level-ma...,Co Founder and CTO - VP of Product Development,As one of the fastest growing online marketing...,Nov 2009 â€“ Present,University of Michigan,https://www.linkedin.com/school/18633/?legacyS...,Bachelor of Science (B.S.),Architecture,1999 â€“ 2004,Walled Lake Central High School,https://www.linkedin.com/school/3204390/?legac...,High School Diploma,NaN,1995 â€“ 1999,nominative@pro,Mr,NaN,www.kidsdraw.org,NaN,NaN,http://www.kidsdraw.org/donate,"Web Design, Marketing, Business Strategy, SEO,...",Web Design,99,Marketing,66,Business Strategy,11,SEO,47,Web Development,40,Online Marketing,32,https://www.linkedin.com/in/jonbowerman,jonbowerman,2022-01-03T20:01:43.218Z,"West Bloomfield, MI",NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.linkedin.com/in/parkeralynch,5. Purchased Membership,Hunter,Parker@HedgeHogHealth.com,NaN,CEO at HedgeHog Health | 2020-2021 Global Educ...,"Royal Oak, Michigan, United States",Parker,Lynch,Parker Lynch,2549.00,22364061.00,NaN,parker@hedgehoghealth.com,HedgeHog Health,https://www.linkedin.com/company/hedgehoghealth/,Chief E

In [5]:
df.sample(5)

,linkedinProfile,Result,Other,email,description,headline,location,firstName,lastName,fullName,subscribers,userId,mutualConnectionsText,mailFromDropcontact,company,companyUrl,jobTitle,jobDescription,jobLocation,jobDateRange,company2,companyUrl2,jobTitle2,jobDescription2,jobDateRange2,school,schoolUrl,schoolDegree,schoolDegreeSpec,schoolDateRange,school2,schoolUrl2,schoolDegree2,schoolDegreeSpec2,schoolDateRange2,qualificationFromDropContact,civilityFromDropContact,phoneNumberFromDropContact,websiteFromDropContact,twitter,website,companyWebsite,allSkills,skill1,endorsement1,skill2,endorsement2,skill3,endorsement3,skill4,endorsement4,skill5,endorsement5,skill6,endorsement6,baseUrl,profileId,timestamp,jobLocation2,connectionsCount,mail,birthday,schoolDescription2,naf5CodeFromDropContact,naf5DesFromDropContact,sirenFromDropContact,siretFromDropContact,siretAddressFromDropContact,siretZipFromDropContact,vatFromDropContact,schoolDescription,nbEmployeesFromDropContact,Unnamed: 72,Unnamed: 73
156,https://www.linkedin.com/in/alexpurtell,2. Sent Personal f/u,Hunter,alex@horizontwolabs.com,NaN,Partner & Growth Lead at Horizon Two Labs,"Columbus, Ohio, United States",Alex,Purtell,Alex Purtell,3304.00,189407345.00,NaN,alex@horizontwolabs.com,Horizon Two Labs,https://www.linkedin.com/company/horizontwolabs/,Partner + Accelerator Director,At Horizon Two Labs we define the adjacent pos...,"Columbus, Ohio Area",Aug 2018 â€“ Present,Rekovo,https://www.linkedin.com/company/rekovo/,Co-founder & CEO,Rekovo is SAAS platform that uses real-time do...,Jan 2015 â€“ Present,The Ohio State University,https://www.linkedin.com/school/19133/?legacyS...,NaN,Entrepreneurial and Small Business Operations,NaN,NaN,NaN,NaN,NaN,NaN,nominative@pro,Mr,NaN,www.horizontwolabs.com,alexpurtell,NaN,http://horizontwolabs.com,"Entrepreneurship, Start-ups, Marketing, Online...",Entrepreneurship,46,Start-ups,34,Marketing,29,Online Advertising,15,Business Development,9,Product Development,7,https://www.linkedin.com/in/alexpurtell,alexpurtell,2022-01-03T22:37:31.540Z,"Columbus, Ohio Area",NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
337,https://www.linkedin.com/in/levi-baker-a54610109/,2021 Member,NaN,NaN,NaN,Builder at Creative elements Inc,"Heber City, Utah, United States",Levi,Baker,Levi Baker,34.00,458370784.00,NaN,NaN,Creative elements Inc,NaN,Builder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mr,NaN,www.thecreativeelements.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.linkedin.com/in/levi-baker-a54610109,levi-baker-a54610109,2022-01-19T14:17:58.548Z,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,https://www.linkedin.com/in/johannesariens,3. Approved,Hollis,ja@routeline.com,If anything Iâ€™m up to sounds interesting or ...,CEO at Route Line - Create & Execute,Greater Seattle Area,Johannes,Ariens,Johannes Ariens,1239.00,137417395.00,NaN,ja@routeline.com,Route Line,https://www.linkedin.com/company/route-line/,CEO,Route Line is an end-to-end adventure company ...,"Seattle, Washington, United States",Jul 2021 â€“ Present,Commodity Project,https://www.linkedin.com/company/commodityproj...,Principal,The Commodity Project is a platform brand and ...,Jan 2021 â€“ Present,Harvard Business School,https://www.linkedin.com/school/18484/?legacyS...,NaN,Alternative Investments,2021 â€“ 2021,University of Washington,https://www.linkedin.com/school/19657/?legacyS...,Professional Certificate Program,Commercial Real Estate Development,2015 â€“ 2016,nominative@pro,Mr,NaN,www.routeline.com,RadifyDev,NaN,http://www.routeline.com,"Construction, Submittals, Construction Managem...",Construction,36,Submittals,23,Construction Management,18,LEED,17,Project Management,15,LEED AP,13,https://www.linkedin.com/in/johannesariens,johannesariens,2022-01-03T20:28:57.663Z,"Seattle, WA",500.00,NaN,2023-06-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
281,https://www.linkedin.com/in/divatommei/,2021 Memb

In [6]:
df.duplicated().sum()

0

We have 348 LinkedIn profiles, with no duplicates at least at first glance. A lot of missing data with no way of filling in the gaps, so we'll take a closer look on how it's distributed in our target groups, as well as look for potential categorical data points.

## Examining data

In [7]:
df.Result.value_counts()

2021 Member                157
2. Sent Personal f/u       127
3. Approved                 28
Out of Sequence             26
4.Purchased Event            6
5. Purchased Membership      4
Name: Result, dtype: int64

So if we count both ideal and second best outcomes our conversion rate is ~3% which sounds normal for an outreach, however it leaves us with very little data; so we'll have to examine past members too as they have at some point paid for a membership - why they are no longer doing so is another question (assuming they have not indeed renewed their membership).

In [12]:
won = df[(df['Result']=='5. Purchased Membership') | (df['Result']=='4.Purchased Event')]
won

,linkedinProfile,Result,Other,email,description,headline,location,firstName,lastName,fullName,subscribers,userId,mutualConnectionsText,mailFromDropcontact,company,companyUrl,jobTitle,jobDescription,jobLocation,jobDateRange,company2,companyUrl2,jobTitle2,jobDescription2,jobDateRange2,school,schoolUrl,schoolDegree,schoolDegreeSpec,schoolDateRange,school2,schoolUrl2,schoolDegree2,schoolDegreeSpec2,schoolDateRange2,qualificationFromDropContact,civilityFromDropContact,phoneNumberFromDropContact,websiteFromDropContact,twitter,website,companyWebsite,allSkills,skill1,endorsement1,skill2,endorsement2,skill3,endorsement3,skill4,endorsement4,skill5,endorsement5,skill6,endorsement6,baseUrl,profileId,timestamp,jobLocation2,connectionsCount,mail,birthday,schoolDescription2,naf5CodeFromDropContact,naf5DesFromDropContact,sirenFromDropContact,siretFromDropContact,siretAddressFromDropContact,siretZipFromDropContact,vatFromDropContact,schoolDescription,nbEmployeesFromDropContact,Unnamed: 72,Unnamed: 73
0,https://www.linkedin.com/in/mark-kohoot-96a08b1,5. Purchased Membership,Hunter,mk@aeroscena.com,Mark founded Aeroscena in 2010 with the missio...,Bio-pharmaceutical / functional fragrance,"Cleveland, Ohio, United States",Mark,Kohoot,Mark Kohoot,1704.00,3172258.00,1 mutual connection: Eric Sullivan,mk@aeroscena.com,Aeroscena,https://www.linkedin.com/company/aeroscena/,CEO,Aeroscena develops and markets fragrances from...,"Cleveland/Akron, Ohio Area",Sep 2010 â€“ Present,Cleveland Clinic Wellness Institute,https://www.linkedin.com/company/cleveland-cli...,Advisor to the Chairman,Supporting the Chairman of the Wellness Instit...,2009 â€“ 2010,Audencia Nantes Ecole de Management,https://www.linkedin.com/school/19998/?legacyS...,master,marketing,1984 â€“ 1985,The Ohio State University,https://www.linkedin.com/school/19133/?legacyS...,BSBA,International Business,1979 â€“ 1983,nominative@pro,Mr,+1 800-671-1890,www.aeroscena.com,ShopAscents,stumbleupon.com/content/1LSMVB,https://aeroscena.com,"Strategic Planning, Leadership, Marketing Stra...",Strategic Planning,31,Leadership,26,Marketing Strategy,24,Strategy,21,New Business Development,18,Business Strategy,15,https://www.linkedin.com/in/mark-kohoot-96a08b1,mark-kohoot-96a08b1,2022-01-03T20:00:58.586Z,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.linkedin.com/in/jonbowerman,5. Purchased Membership,Hunter,jon@kidsdraw.org,A creative problem solver who can rapidly desi...,Co Founder and CTO - Product Development and M...,"Farmington, Michigan, United States",Jon,Bowerman,Jon Bowerman,1745.00,14910006.00,1 mutual connection: Eric Sullivan,jon@kidsdraw.org,Kids Draw Bob,https://www.linkedin.com/company/kidsdrawbob/,President and Illustrator,The energy in a room of children who've found ...,"Bloomfield Hills, MI",Jul 2019 â€“ Present,"High Level Marketing, LLC",https://www.linkedin.com/company/high-level-ma...,Co Founder and CTO - VP of Product Development,As one of the fastest growing online marketing...,Nov 2009 â€“ Present,University of Michigan,https://www.linkedin.com/school/18633/?legacyS...,Bachelor of Science (B.S.),Architecture,1999 â€“ 2004,Walled Lake Central High School,https://www.linkedin.com/school/3204390/?legac...,High School Diploma,NaN,1995 â€“ 1999,nominative@pro,Mr,NaN,www.kidsdraw.org,NaN,NaN,http://www.kidsdraw.org/donate,"Web Design, Marketing, Business Strategy, SEO,...",Web Design,99,Marketing,66,Business Strategy,11,SEO,47,Web Development,40,Online Marketing,32,https://www.linkedin.com/in/jonbowerman,jonbowerman,2022-01-03T20:01:43.218Z,"West Bloomfield, MI",NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.linkedin.com/in/parkeralynch,5. Purchased Membership,Hunter,Parker@HedgeHogHealth.com,NaN,CEO at HedgeHog Health | 2020-2021 Global Educ...,"Royal Oak, Michigan, United States",Parker,Lynch,Parker Lynch,2549.00,22364061.00,NaN,parker@hedgehoghealth.com,HedgeHog Health,https://www.linkedin.com/company/hedgehoghealth/,Chief E

In [13]:
churned = df[df['Result']=='2021 Member']

In [19]:
churned.sample(5)

,linkedinProfile,Result,Other,email,description,headline,location,firstName,lastName,fullName,subscribers,userId,mutualConnectionsText,mailFromDropcontact,company,companyUrl,jobTitle,jobDescription,jobLocation,jobDateRange,company2,companyUrl2,jobTitle2,jobDescription2,jobDateRange2,school,schoolUrl,schoolDegree,schoolDegreeSpec,schoolDateRange,school2,schoolUrl2,schoolDegree2,schoolDegreeSpec2,schoolDateRange2,qualificationFromDropContact,civilityFromDropContact,phoneNumberFromDropContact,websiteFromDropContact,twitter,website,companyWebsite,allSkills,skill1,endorsement1,skill2,endorsement2,skill3,endorsement3,skill4,endorsement4,skill5,endorsement5,skill6,endorsement6,baseUrl,profileId,timestamp,jobLocation2,connectionsCount,mail,birthday,schoolDescription2,naf5CodeFromDropContact,naf5DesFromDropContact,sirenFromDropContact,siretFromDropContact,siretAddressFromDropContact,siretZipFromDropContact,vatFromDropContact,schoolDescription,nbEmployeesFromDropContact,Unnamed: 72,Unnamed: 73
306,https://www.linkedin.com/in/jayfaires/,2021 Member,NaN,NaN,"Faires created TWA (The Wellness Agency), whic...",The Wellness Agency (TWA),"Los Angeles County, California, United States",Jay,Faires,Jay Faires,2564.00,16437770.00,NaN,NaN,The Wellness Agency,https://www.linkedin.com/company/the-wellness-...,Founder,The Wellness Agency (TWA) empowers wellness bu...,"Venice, Ca",Jan 2018 â€“ Present,Lionsgate,https://www.linkedin.com/company/lionsgate/,President of Music,NaN,2005 â€“ 2010,Duke University,https://www.linkedin.com/school/18765/?legacyS...,MBA,NaN,1985 â€“ 1987,Sewanee: The University of the South,NaN,"Bachelor of Arts, with Honors",Psychology,1981 â€“ 1985,NaN,Mr,NaN,www.thewellness.agency,docroc_jay,angel.co/jay-faires,http://www.thewellness.agency/,"Entrepreneurship, Television, Entertainment, S...",Entrepreneurship,0,Television,0,Entertainment,0,Social Media,0,Strategic Partnerships,0,New Media,0,https://www.linkedin.com/in/jayfaires,jayfaires,2022-01-19T13:52:40.395Z,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
265,https://www.linkedin.com/in/chriskresser/,2021 Member,NaN,chris@kresserinstitute.com,"Chris Kresser, M.S., L.Ac, is a practitioner o...",Co-Founder and President at California Center ...,"Oakland, California, United States",Chris,Kresser,Chris Kresser,2963.00,68898932.00,NaN,NaN,Kresser Institute,NaN,Founder and CEO,NaN,"Las Vegas, Nevada Area",Oct 2015 â€“ Present,California Center for Functional Medicine,https://www.linkedin.com/company/california-ce...,Co-Founder and President,NaN,Oct 2014 â€“ Present,"Acupuncture & Integrative Medicine College, Be...",NaN,MA,Acupuncture and Oriental Medicine,2007 â€“ 2010,UC Berkeley,NaN,BA,Communications & Social Change,1992 â€“ 1996,NaN,Mr,NaN,www.kresserinstitute.com,chriskresser,thehealthyskeptic.org,NaN,"Nutrition, Holistic Health, Blogging, Wellness...",Nutrition,99+,Holistic Health,99+,Blogging,99+,Wellness,99+,Healing,99+,Wellness Coaching,65,https://www.linkedin.com/in/chriskresser,chriskresser,2022-01-19T13:19:48.978Z,"Berkeley, CA",NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chris@kresserinstitute.com
288,https://www.linkedin.com/in/eric-rieger-a559a010/,2021 Member,NaN,NaN,CRNA - In partnership with gastroenterologist ...,Sales & Market Dev @ Atrantil - Ambassador of ...,Dallas-Fort Worth Metroplex,Eric,Rieger,Eric Rieger,393.00,38280161.00,NaN,NaN,Atrantil,https://www.linkedin.com/company/atrantil/,PR-Sales and Market Development,"Developed by gastroenterologist Ken Brown, MD ...","Dallas, TX",Feb 2015 â€“ Present,Tequila 512 - Willis Importing,NaN,Sales Consultant,Successful launch of Tequila 512 into the Texa...,2012 â€“ Present,Texas Tech University,https://www.linkedin.com/school/19505/?legacyS...,BS,Zoology/Chemistry,1994 â€“ 1998,Texas Wesleyan University,https://www.linkedin.com/school/19510/?legacyS...,MS,Anesthesia,2006 â€“ 2008,NaN,Mr,NaN,www.atrantil.com,NaN,tequila512.com,http://atrantil.com,"Healthcare, Customer Service, Hospi

Seeing a lot of: "head honchos" (founder/owner/C-level etc.), also first-name email addresses. Subscribers, twitter, education and skills seem to vary a lot, as well as age and location. All seem to be within US, suggesting this was a US event.  
Let's take a look at the rest and see if any of that is different.

In [20]:
lost = df[(df.index.isin(won.index)==False) & (df.index.isin(churned.index)==False)]

In [30]:
lost.sample(5)

,linkedinProfile,Result,Other,email,description,headline,location,firstName,lastName,fullName,subscribers,userId,mutualConnectionsText,mailFromDropcontact,company,companyUrl,jobTitle,jobDescription,jobLocation,jobDateRange,company2,companyUrl2,jobTitle2,jobDescription2,jobDateRange2,school,schoolUrl,schoolDegree,schoolDegreeSpec,schoolDateRange,school2,schoolUrl2,schoolDegree2,schoolDegreeSpec2,schoolDateRange2,qualificationFromDropContact,civilityFromDropContact,phoneNumberFromDropContact,websiteFromDropContact,twitter,website,companyWebsite,allSkills,skill1,endorsement1,skill2,endorsement2,skill3,endorsement3,skill4,endorsement4,skill5,endorsement5,skill6,endorsement6,baseUrl,profileId,timestamp,jobLocation2,connectionsCount,mail,birthday,schoolDescription2,naf5CodeFromDropContact,naf5DesFromDropContact,sirenFromDropContact,siretFromDropContact,siretAddressFromDropContact,siretZipFromDropContact,vatFromDropContact,schoolDescription,nbEmployeesFromDropContact,Unnamed: 72,Unnamed: 73
11,https://www.linkedin.com/in/anne-zizzo-0075603,3. Approved,Hunter,NaN,"At Zizzo Group, we believe that successful eng...",Founder & CEO at Zizzo Group - Engagement Mark...,"Milwaukee, Wisconsin, United States",Anne,Zizzo,Anne Zizzo,2775.00,10326535.00,NaN,NaN,"Zizzo Ventures, LLC",NaN,Founder,"Investment, leasing, development","Milwaukee, Wisconsin, United States",May 2021 â€“ Present,Zizzo Group - Engagement Marketing,https://www.linkedin.com/company/zizzo-group-m...,Founder & CEO,NaN,Aug 1995 â€“ Present,Marquette University,https://www.linkedin.com/school/19675/?legacyS...,BA,Journalism,1983 â€“ 1987,NaN,NaN,NaN,NaN,NaN,NaN,Mrs,NaN,www.sanford-artedventures.com,NaN,zizzogroup.com,NaN,"Public Relations, Integrated Marketing, Social...",Public Relations,99+,Integrated Marketing,99+,Social Media,97,Advertising,49,Media Relations,39,Strategic Communications,29,https://www.linkedin.com/in/anne-zizzo-0075603,anne-zizzo-0075603,2022-01-03T20:11:36.502Z,"207 N. Milwaukee Street, Milwaukee, WI 53202",500.00,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,https://www.linkedin.com/in/samantha-friedman-...,3. Approved,Hunter,NaN,NaN,"Manager, Marketing","Birmingham, Michigan, United States",Samantha,Friedman,Samantha Friedman,1176.00,72697048.00,NaN,NaN,The Mom Project,https://www.linkedin.com/company/the-mom-project/,Seller Marketing Manager @ Etsy,NaN,"Brooklyn, New York, United States",Jul 2021 â€“ Present,Fuse45,https://www.linkedin.com/company/fuse45/,Co Founder & Co Owner,FUSE LIVE is a uniquely sustainable and full-b...,Jun 2015 â€“ Present,University of Michigan - Stephen M. Ross Schoo...,https://www.linkedin.com/school/18634/?legacyS...,Bachelor's of Business Administration,Marketing,2006 â€“ 2010,University of Michigan - Stephen M. Ross Schoo...,https://www.linkedin.com/school/18634/?legacyS...,BBA,Marketing,2006 â€“ 2010,NaN,Mrs,NaN,www.themomproject.com,Fuse45RoyalOak,watch.fuselive.net/,http://themomproject.com,"Marketing Communications, Marketing, Marketing...",Marketing Communications,24,Marketing,16,Marketing Strategy,10,Integrated Marketing,10,Event Management,8,New Business Development,5,https://www.linkedin.com/in/samantha-friedman-...,samantha-friedman-07441a20,2022-01-03T20:14:54.828Z,NaN,NaN,NaN,NaT,2009 Receipient of BBA Entrepreneur of the Yea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,https://www.linkedin.com/in/schelkidd,2. Sent Personal f/u,Hunter,NaN,Two words can describe Larschelby; Passion & E...,Independent Boisset Collection Ambassador at B...,"Kohler, Wisconsin, United States",Larschelby Kidd,WisconsinWineGuy,Larschelby Kidd WisconsinWineGuy,772.00,83909867.00,NaN,NaN,Boisset Collection,https://www.linkedin.com/company/boisset-colle...,Independent Boisset Collection Ambassador,I am a Independent Boisset Collection Ambassad...,Wisconsin,Jan 2019 â€“ Present,Aras Promotions,https://www.linkedin.com/company/aras-promotions/,Brand Ambassador,"I demonstrate, educate and sample to customers...",Mar 2018 â€“

A quick look confirms the impression of the outreach being targeted at business leaders, superficially there is little to no difference between the profiles. Next we'll pick a few characteristicts to compare won, churned and lost members/attendees to see if we can detect any patterns.

In [102]:
good_combined = pd.concat([won,churned])

In [103]:
dfs = (('won',won), ('churned',churned), ('combined positive', good_combined), ('lost',lost)) #for mass processing

## Detecting patterns

In [104]:
for frame in dfs:
    print(f'Examining {frame[0]} leads.')
    print(f'Mean subscribers: {frame[1].subscribers.mean()}, median subscribers: {frame[1].subscribers.median()}')
    print(f'Mean user id: {frame[1].userId.mean()}')
    print('='*100)

Examining won leads.
Mean subscribers: 2504.7, median subscribers: 1700.0
Mean user id: 99719454.2
Examining churned leads.
Mean subscribers: 2621.121794871795, median subscribers: 1470.5
Mean user id: 114916101.24489796
Examining combined positive leads.
Mean subscribers: 2614.10843373494, median subscribers: 1525.0
Mean user id: 113509004.2962963
Examining lost leads.
Mean subscribers: 2628.1215469613257, median subscribers: 1175.0
Mean user id: 152985565.2872928


We have little numerical data - user ids are barely that, although they might suggest that more seasoned LinkedIn users (= smaller id) tend to be more receptive to the events and membership.  
While mean number of subscribers is almost identical across the three groups, medians vary significantly - with larger numbers indicating a propensity towards membership or at least attending an event.  
Next we'll quantify what we can and mine for possible categorical data.

In [105]:
for frame in dfs:   
    frame[1]['desc_len'] = frame[1].description.apply(lambda x: 0 if x is np.nan else len(x))
    frame[1]['headline_len'] = frame[1].headline.apply(lambda x: 0 if x is np.nan else len(x))
    print(f'Examining {frame[0]} leads.')
    print(f'Mean length of headline: {frame[1].headline_len.mean()}, median length of headline: {frame[1].headline_len.median()}')
    print(f'Mean length of description: {frame[1].desc_len.mean()}, median length of description: {frame[1].desc_len.median()}')

Examining won leads.
Mean length of headline: 62.0, median length of headline: 50.0
Mean length of description: 570.7, median length of description: 398.5
Examining churned leads.
Mean length of headline: 65.59235668789809, median length of headline: 49.0
Mean length of description: 756.6751592356688, median length of description: 432.0
Examining combined positive leads.
Mean length of headline: 65.37724550898204, median length of headline: 49.0
Mean length of description: 745.5389221556886, median length of description: 432.0
Examining lost leads.
Mean length of headline: 59.005524861878456, median length of headline: 50.0
Mean length of description: 684.5911602209944, median length of description: 440.0


The length of sections themselves shows little variabiltiy, although it's curious that won and previously won leads show a slight tendency to write longer headlines and shorter descriptions. Next we'll look at contents of descriptions and headlines to see if there's any difference in branding oneself that correlates with purchasing membership or an event ticket.

In [106]:
def inspect_titles(text):
    titles = (('owner','owner'),
        ('founder|co-founder','founder'),
        ('ceo| chief executive','ceo'),
        ('c[a-z]{2}| chief .+ officer','c-level'),
        ('entrepren','entrepreneur'),
        ('managing [a-z]+| director| s*vp| head of ','higher_management'))
    
    title_list = []
    for pair in titles:
        if text is not np.nan:
            if re.search(pair[0], text.lower()):
                title_list.append(pair[1])
    return title_list

In [107]:
for frame in dfs:
    frame[1]['title_list_desc']= frame[1].description.apply(inspect_titles)
    frame[1]['title_list_headline']= frame[1].headline.apply(inspect_titles)
    frame[1]['desc_titles'] = frame[1].title_list_desc.apply(lambda x: len(x))
    frame[1]['headline_titles'] = frame[1].title_list_headline.apply(lambda x: len(x))
    print(f'Examining {frame[0]} leads.')
    print(f'Mean number of titles in headline: {frame[1].headline_titles.mean()}, median number of titles in headline: {frame[1].headline_titles.median()}')
    print(f'Mean number of titles in description: {frame[1].desc_titles.mean()}, median number of titles in description: {frame[1].desc_titles.median()}')

Examining won leads.
Mean number of titles in headline: 1.7, median number of titles in headline: 1.5
Mean number of titles in description: 1.1, median number of titles in description: 1.0
Examining churned leads.
Mean number of titles in headline: 1.5987261146496816, median number of titles in headline: 1.0
Mean number of titles in description: 1.5286624203821657, median number of titles in description: 1.0
Examining combined positive leads.
Mean number of titles in headline: 1.6047904191616766, median number of titles in headline: 1.0
Mean number of titles in description: 1.5029940119760479, median number of titles in description: 1.0
Examining lost leads.
Mean number of titles in headline: 1.4364640883977902, median number of titles in headline: 1.0
Mean number of titles in description: 1.4751381215469612, median number of titles in description: 1.0


In [108]:
for col in df.columns:
    for frame in dfs:
        print(f'Leads: {frame[0]}, missing values in {col}: {round(((frame[1][col].isna().sum())/frame[1].shape[0]),2)*100}%')
    print('='*100)

Leads: won, missing values in linkedinProfile: 0.0%
Leads: churned, missing values in linkedinProfile: 0.0%
Leads: combined positive, missing values in linkedinProfile: 0.0%
Leads: lost, missing values in linkedinProfile: 0.0%
Leads: won, missing values in Result: 0.0%
Leads: churned, missing values in Result: 0.0%
Leads: combined positive, missing values in Result: 0.0%
Leads: lost, missing values in Result: 0.0%
Leads: won, missing values in Other: 0.0%
Leads: churned, missing values in Other: 100.0%
Leads: combined positive, missing values in Other: 94.0%
Leads: lost, missing values in Other: 0.0%
Leads: won, missing values in email: 30.0%
Leads: churned, missing values in email: 36.0%
Leads: combined positive, missing values in email: 35.0%
Leads: lost, missing values in email: 40.0%
Leads: won, missing values in description: 30.0%
Leads: churned, missing values in description: 25.0%
Leads: combined positive, missing values in description: 26.0%
Leads: lost, missing values in descr

Doesn't look like gaps in any of the aspects of profiles can tell us anything meaningful about the difference between the groups.

In [109]:
for frame in dfs:
    frame[1]['jobDateRange2'] = frame[1]['jobDateRange2'].fillna(' ')
    frame[1]['two_jobs'] = frame[1].jobDateRange2.apply(lambda x: True if 'Present' in str(x) else False)
    print(f'Examining {frame[0]} leads.')
    print(f'Share of people combining jobs: {round((frame[1].two_jobs.sum()/frame[1].shape[0]),2)}')

Examining won leads.
Share of people combining jobs: 0.9
Examining churned leads.
Share of people combining jobs: 0.6
Examining combined positive leads.
Share of people combining jobs: 0.62
Examining lost leads.
Share of people combining jobs: 0.7


It appears that having two jobs at once used to somewhat reduce the chance of becoming a member; however it's not yet clear how significant this correlation is.

## Statistical significance

A couple of differences we were able to spot warrant running test of statistical significance on. We will be comparing two groups of profiles: those who purchased membership or a ticket event + those who were members in the past, and those who failed to do either of those after the last outreach.  
We'll begin with the number of subscribers - we suspect that people cultivating a larger number of followers on LinkedIn are more likely to become members and attend events as the prime networking and self-promoting opportunities.  
We'll start with a formal check of normality of distribution using the Shapiro-Wilk test, H0 being that the data is distributed normally, and H1 being that the distribution is not normal.

In [113]:
alpha = 0.01
dfs_2 = (('combined positive', good_combined), ('lost',lost))

In [120]:
for frame in dfs_2:
    print(f'Checking distribution in {frame[0]}:')
    p_val = st.shapiro(frame[1].subscribers.dropna()).pvalue
    if  p_val > alpha:
        print(f'P-value: {p_val}. Can not reject the null hypothesis, the number of subscribers is distributed normally.')
    else:
        print(f'P-value: {p_val}. The null hypothesis should be rejected, the number of subscribers is distributed not normally.')

Checking distribution in combined positive:
P-value: 1.1725375345625936e-17. The null hypothesis should be rejected, the number of subscribers is distributed not normally.
Checking distribution in lost:
P-value: 1.0155986513698732e-22. The null hypothesis should be rejected, the number of subscribers is distributed not normally.


As the data is not distributed normally we will proceed with using the The Mann-Whitney U test to check whether the observed difference is significant.  
H0: the average number of subscribers in both groups is the same.  
H1: the average number of subscribers in the 'good_combined' group is higher.

In [123]:
x = good_combined.subscribers.dropna()
y = lost.subscribers.dropna()
p_val = st.mannwhitneyu(x,y, alternative='greater').pvalue
if  p_val < alpha:
    print(f'P-value: {p_val}. We reject the null hypothesis, the average number of subsribers in "good_combined" is higher.')
else:
    print(f'P-value: {p_val}. We can not reject the null hypothesis, there is no significant difference between the numbers of subscribers in two groups.')

P-value: 0.09707386827171183. We can not reject the null hypothesis, there is no significant difference between the numbers of subscribers in two groups.
